In [2]:
import json
import re

# 이전 버전 (v2) - 비교를 위해 포함
def parse_prediction_v2(pred_str: str) -> str:
    if not isinstance(pred_str, str): return 'parsing_failed'
    try:
        clean_str = pred_str
        if '```json' in clean_str: clean_str = clean_str.split('```json')[1].split('```')[0]
        elif '```' in clean_str: clean_str = clean_str.split('```')[1].split('```')[0]
        clean_str = clean_str.strip()
        start_brace, end_brace = clean_str.find('{'), clean_str.rfind('}')
        if start_brace != -1 and end_brace != -1 and start_brace < end_brace:
            json_part = clean_str[start_brace : end_brace + 1]
            try:
                data = json.loads(json_part)
                category = data.get('category')
                if category in ['violence', 'normal']: return category
            except json.JSONDecodeError: pass
        cat_match = re.search(r'["\']category["\']\s*:\s*["\'](violence|normal)["\']', clean_str)
        if cat_match: return cat_match.group(1)
        return 'no_json_found'
    except Exception: return 'parsing_failed'

# 대소문자 무시 버전 (v3)
def parse_prediction_v3(pred_str: str) -> str:
    if not isinstance(pred_str, str): return 'parsing_failed'
    try:
        clean_str = pred_str
        if '```json' in clean_str: clean_str = clean_str.split('```json')[1].split('```')[0]
        elif '```' in clean_str: clean_str = clean_str.split('```')[1].split('```')[0]
        clean_str = clean_str.strip()
        start_brace, end_brace = clean_str.find('{'), clean_str.rfind('}')
        if start_brace != -1 and end_brace != -1 and start_brace < end_brace:
            json_part = clean_str[start_brace : end_brace + 1]
            try:
                data = json.loads(json_part)
                category = data.get('category')
                if isinstance(category, str) and category.lower() in ['violence', 'normal']:
                    return category.lower()
            except json.JSONDecodeError: pass
        cat_match = re.search(r'["\']category["\']\s*:\s*["\'](violence|normal)["\']', clean_str, re.IGNORECASE)
        if cat_match: return cat_match.group(1).lower()
        return 'no_json_found'
    except Exception: return 'parsing_failed'


# ----------------- 테스트 케이스 정의 (대소문자 케이스 추가) -----------------
test_cases = [
    ("정상 (소문자)", '{"category": "normal", "description": "..."}'),
    ("정상 (소문자)", '{"category": "violence", "description": "..."}'),
    ("대소문자: Title Case", '{"category": "Violence", "description": "..."}'),
    ("대소문자: ALL CAPS", '{"category": "NORMAL", "description": "..."}'),
    ("대소문자: MiXeD CaSe", '{"category": "vIoLeNcE", "description": "..."}'),
    ("대소문자: Regex Fallback", "'category': 'nOrMaL', 'description': '...'"),
    ("마크다운 포함", '```json\n{"category": "normal", "description": "..."}\n```'),
    ("앞뒤 텍스트 포함", 'Answer: {"category": "violence", "description": "..."}'),
    ("JSON 형식 깨짐", '{"category": "normal" "description": "..."}'),
    ("category 키 없음", '{"action": "running", "description": "..."}'),
    ("완전한 쓰레기값", "A cat playing with a ball."),
    ("빈 문자열", ""),
]

# ----------------- 테스트 실행 및 결과 비교 -----------------
print(f"{'테스트 설명':<25} | {'V2 결과 (기존)':<15} | {'V3 결과 (수정)':<15}")
print("-" * 65)

for desc, case in test_cases:
    res2 = parse_prediction_v2(case)
    res3 = parse_prediction_v3(case)
    print(f"{desc:<25} | {res2:<15} | {res3:<15}")

테스트 설명                    | V2 결과 (기존)      | V3 결과 (수정)     
-----------------------------------------------------------------
정상 (소문자)                  | normal          | normal         
정상 (소문자)                  | violence        | violence       
대소문자: Title Case          | no_json_found   | violence       
대소문자: ALL CAPS            | no_json_found   | normal         
대소문자: MiXeD CaSe          | no_json_found   | violence       
대소문자: Regex Fallback      | no_json_found   | normal         
마크다운 포함                   | normal          | normal         
앞뒤 텍스트 포함                 | violence        | violence       
JSON 형식 깨짐                | normal          | normal         
category 키 없음             | no_json_found   | no_json_found  
완전한 쓰레기값                  | no_json_found   | no_json_found  
빈 문자열                     | no_json_found   | no_json_found  


In [ ]:
import json, re

def parse_prediction(pred_str: str) -> str:
    try:
        clean_str = pred_str
        if '```json' in clean_str:
            clean_str = clean_str.split('```json')[1].split('```')[0]
        elif '```' in clean_str:
            clean_str = clean_str.split('```')[1].split('```')[0]
        clean_str = clean_str.strip()
        start_brace = clean_str.find('{')
        end_brace = clean_str.rfind('}')
        if start_brace != -1 and end_brace != -1 and start_brace < end_brace:
            json_part = clean_str[start_brace:end_brace + 1]
            data = json.loads(json_part)
            category = data.get('category')
            if category in ['violence', 'normal']:
                return category
        cat_match = re.search(r'["\']category["\']\s*:\s*["\'](violence|normal)["\']', clean_str)
        if cat_match:
            return cat_match.group(1)
        return 'violence'
    except Exception:
        return 'violence'
    
    
    